# License Plate Detection Training with YOLOv8 (CPU Version)

Notebook này huấn luyện model YOLOv8 để phát hiện vị trí biển số xe, tối ưu cho CPU

In [ ]:
# Cài đặt thư viện cần thiết
%pip install ultralytics

from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import yaml
import os

# Kiểm tra thiết bị
device = 'cpu'  # Force CPU
print(f"Training on: {device}")


Using CUDA: False


## Kiểm tra Dataset
Xác nhận cấu trúc dataset trước khi training

In [ ]:
# Load và kiểm tra cấu hình dataset
yaml_path = '../training/LP_detection.yaml'

with open(yaml_path, 'r') as f:
    cfg = yaml.safe_load(f)

print("Dataset configuration:")
print(f"- Path: {cfg['path']}")
print(f"- Train: {cfg['train']}")
print(f"- Val: {cfg['val']}")

# Kiểm tra số lượng ảnh
train_path = os.path.join(cfg['path'], cfg['train'])
val_path = os.path.join(cfg['path'], cfg['val'])

train_images = len([f for f in os.listdir(train_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
val_images = len([f for f in os.listdir(val_path) if f.endswith(('.jpg', '.png', '.jpeg'))])

print(f"\nSố lượng ảnh:")
print(f"- Training: {train_images} ảnh")
print(f"- Validation: {val_images} ảnh")

Dataset configuration:
- Path: ../images
- Train: train
- Val: val
- Classes: ['license_plate']

Số lượng ảnh:
- Training: 3433 ảnh
- Validation: 1145 ảnh


## Training Model
Huấn luyện YOLOv8 với cấu hình tối ưu cho CPU

In [ ]:
# Tạo model YOLOv8n mới
model = YOLO('yolov8n.pt')

# Train với cấu hình tối ưu cho CPU
results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    batch=8,  # Giảm batch size cho CPU
    device='cpu',
    name='LP_detector_v8_cpu',
    cache=True,  # Cache images để tăng tốc training
    # Data augmentation (giảm để phù hợp với CPU)
    hsv_h=0.015,
    hsv_s=0.5,
    hsv_v=0.3,
    degrees=10.0,
    translate=0.2,
    scale=0.3,
    shear=2.0,
    perspective=0.0005,
    flipud=0.01,
    fliplr=0.3,
    mosaic=0.5,  # Giảm mosaic aug
    mixup=0.0,   # Tắt mixup để giảm tải CPU
    copy_paste=0.0,  # Tắt copy-paste để giảm tải CPU
    # Tham số huấn luyện
    warmup_epochs=3,
    patience=20,
    lr0=0.01,
    lrf=0.001,
    momentum=0.937,
    weight_decay=0.0005,
    # Tối ưu cho object detection
    box=7.5,
    cls=0.3,
    dfl=1.5,
    close_mosaic=10,
    # CPU optimization
    workers=1,  # Giảm số workers
    save_period=20  # Lưu checkpoint sau mỗi 20 epochs
)

NameError: name 'YOLO' is not defined

## Training Metrics
Vẽ biểu đồ các metrics trong quá trình training

In [ ]:
# Plot training metrics
plt.figure(figsize=(20, 10))

# Plot losses
plt.subplot(2, 2, 1)
plt.plot(results.results_dict['train/box_loss'], label='train')
plt.plot(results.results_dict['val/box_loss'], label='val')
plt.title('Box Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(results.results_dict['metrics/precision'], label='precision')
plt.plot(results.results_dict['metrics/recall'], label='recall')
plt.title('Precision & Recall')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(results.results_dict['metrics/mAP50(B)'], label='mAP50')
plt.plot(results.results_dict['metrics/mAP50-95(B)'], label='mAP50-95')
plt.title('Mean Average Precision')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

## Validate Model
Đánh giá model trên tập validation

In [ ]:
# Validate model
val_results = model.val()

print("\nValidation Results:")
metrics = [
    'metrics/precision(B)',
    'metrics/recall(B)',
    'metrics/mAP50(B)',
    'metrics/mAP50-95(B)'
]

for metric in metrics:
    if metric in val_results.results_dict:
        print(f"{metric}: {val_results.results_dict[metric]:.4f}")

## Test Model
Kiểm tra model trên một số ảnh test

In [ ]:
# Test model trên ảnh mẫu
test_images = ['../test_image/1.jpg', '../test_image/3.jpg']

for img_path in test_images:
    results = model.predict(
        source=img_path,
        conf=0.25,
        iou=0.45,
        save=True
    )
    
    # Hiển thị kết quả
    plt.figure(figsize=(12, 8))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.title(f'Detection Results - {os.path.basename(img_path)}')
    plt.show()
    
    # In thông tin dự đoán
    print(f"\nResults for {os.path.basename(img_path)}:")
    for box in results[0].boxes:
        print(f"Confidence: {box.conf[0]:.2f}")